# Chapter6-1

## 固有表現認識モデルの実装

### ライブラリのインストール

In [ ]:
!pip install datasets transformers[ja,torch] spacy-alignments seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 65.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━

### データセットの取得

今回は*Chapter6-1*で取得したデータセットを使用する

In [ ]:
from datasets import load_dataset

# データセットを読み込む
dataset = load_dataset("llm-book/ner-wikipedia-dataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for llm-book/ner-wikipedia-dataset contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/ner-wikipedia-dataset
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major rel

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

### トークナイザの取得

In [ ]:
from transformers import AutoTokenizer

# トークナイザを読み込む
model_name = "cl-tohoku/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# トークナイゼーションを行う
subwords = "/".join(tokenizer.tokenize(dataset["train"][0]["text"]))
characters = "/".join(dataset["train"][0]["text"])

print(f"サブワード単位: {subwords}")
print(f"文字単位: {characters}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

サブワード単位: さくら/学院/、/C/##ia/##o/Sm/##ile/##s/の/メンバー/。
文字単位: さ/く/ら/学/院/、/C/i/a/o/ /S/m/i/l/e/s/の/メ/ン/バ/ー/。


### ラベルとIDを対応付けるdictを作成

In [ ]:
from pprint import pprint
import torch

def create_label2id(
    entities_list: list[list[dict[str, str | int]]]
) -> dict[str, int]:

  """ ラベルとIDを紐づけるdictを作成 """

  label2id = {"0": 0}

  # 固有表現タイプのsetを獲得して並び替える
  entity_types = set(
      [e["type"] for entities in entities_list for e in entities]
  )
  entity_types = sorted(entity_types)

  for i, entity_type in enumerate(entity_types):
    label2id[f"B-{entity_type}"] = i * 2 + 1
    label2id[f"I-{entity_type}"] = i * 2 + 2

  return label2id

# ラベルIDを紐づけるdictを作成
label2id = create_label2id(dataset["train"]["entities"])
id2label = {v: k for k, v in label2id.items()}
pprint(label2id)
pprint(id2label)

{'0': 0,
 'B-その他の組織名': 1,
 'B-イベント名': 3,
 'B-人名': 5,
 'B-地名': 7,
 'B-政治的組織名': 9,
 'B-施設名': 11,
 'B-法人名': 13,
 'B-製品名': 15,
 'I-その他の組織名': 2,
 'I-イベント名': 4,
 'I-人名': 6,
 'I-地名': 8,
 'I-政治的組織名': 10,
 'I-施設名': 12,
 'I-法人名': 14,
 'I-製品名': 16}
{0: '0',
 1: 'B-その他の組織名',
 2: 'I-その他の組織名',
 3: 'B-イベント名',
 4: 'I-イベント名',
 5: 'B-人名',
 6: 'I-人名',
 7: 'B-地名',
 8: 'I-地名',
 9: 'B-政治的組織名',
 10: 'I-政治的組織名',
 11: 'B-施設名',
 12: 'I-施設名',
 13: 'B-法人名',
 14: 'I-法人名',
 15: 'B-製品名',
 16: 'I-製品名'}


### データの前処理

上記の処理に基づいてデータの前処理を行う

In [ ]:
from transformers.tokenization_utils_base import BatchEncoding
from transformers import PreTrainedTokenizer
from spacy_alignments.tokenizations import get_alignments

def preprocess_data(
    data: dict[str, any],
    tokenizer: PreTrainedTokenizer,
    label2id: dict[int, str]
) -> BatchEncoding:
  """ データの前処理 """

  # テキストのトークナイゼーション
  inputs = tokenizer(
      data["text"], return_tensors="pt", return_special_tokens_mask=True,
  )
  inputs = {k: v.squeeze(0) for k, v in inputs.items()}

  # 文字のlistとトークンのlistのアライメントをとる
  characters = list(data["text"])
  tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"])
  char_to_token_indices, _ = get_alignments(characters, tokens)

  # "0"のIDのlistを作成する
  labels = torch.zeros_like(inputs["input_ids"])
  for entity in data["entities"]:
    start_token_indices = char_to_token_indices[entity["span"][0]]
    end_token_indices = char_to_token_indices[entity["span"][1] - 1]

    # 文字に対応するトークンが存在しないときスキップ
    if len(start_token_indices) == 0 or len(end_token_indices) == 0:
      continue

    start, end = start_token_indices[0], end_token_indices[0]
    entity_type = entity["type"]

    # 固有表現開始位置に"B-"のIDを設定する
    labels[start] = label2id[f"B-{entity_type}"]

    # 固有表現開始トークン以外の位置に"I-"のIDを設定
    if start != end:
      labels[start + 1 : end + 1] = label2id[f"I-{entity_type}"]

  # 特殊トークンの位置のIDは-100とする
  labels[torch.where(inputs["special_tokens_mask"])] = -100
  inputs["labels"] = labels
  return inputs

# トレーニングセットに対して前処理
train_dataset = dataset["train"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id" : label2id,
    },
    remove_columns=dataset["train"].column_names,
)

# 検証セットに対し
val_dataset = dataset["validation"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id" : label2id
    },
    remove_columns=dataset["validation"].column_names
)

Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, 'label2id': {'0': 0, 'B-その他の組織名': 1, 'I-その他の組織名': 2, 'B-イベント名': 3, 'I-イベント名': 4

Map:   0%|          | 0/4274 [00:00<?, ? examples/s]

Map:   0%|          | 0/534 [00:00<?, ? examples/s]

In [ ]:
# 前処理が完了したデータセットを表示
print(train_dataset.features)
print(val_dataset)

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'special_tokens_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}
Dataset({
    features: ['input_ids', 'token_type_ids', 'special_tokens_mask', 'attention_mask', 'labels'],
    num_rows: 534
})


### モデルの準備

AutoModelForTokenClassificationを使用して実装。

それに加えて、ラベルに関する情報を与える`label2id`, `id2label`を渡す。


In [ ]:
from transformers import (
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
)

# モデルの読み込み
model = AutoModelForTokenClassification.from_pretrained(
    model_name, label2id=label2id, id2label=id2label
)

# collate関数にDataCollaorForTokenClassificationを使用
data_collator = DataCollatorForTokenClassification(tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/447M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### モデルのファインチューニング

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers.trainer_utils import set_seed

set_seed(42)

# Trainerに渡す引数を初期化
train_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Learning_LLM/chapter6/model",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=1e-4,
    lr_scheduler_type="linear",
    warmup_ratio=0.1,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    fp16=True,
)

# Trainerの初期化
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    args=train_args,
)

In [ ]:
# 学習の実行
trainer.train()
trainer.save_model("/content/drive/MyDrive/Learning_LLM/chapter6/result")

Epoch,Training Loss,Validation Loss
1,0.636100,0.102747
2,0.070800,0.092589
3,0.027900,0.092396
4,0.012700,0.100052
5,0.006500,0.102441


## 固有表現の予測・抽出

### 固有表現ラベルの予測


In [ ]:
def convert_list_dict_to_dict_list(
    list_dict: dict[str, list]
) -> list[dict[str, list]]:
  """ ミニバッチのデータを事例単位のlistに変換する """

  dict_list = []

  #キーのリストの作成
  keys = list(list_dict.keys())
  for idx in range(len(list_dict[keys[0]])):
    # dictの各キーからデータを取り出してリストに追加
    dict_list.append({key: list_dict[key][idx] for key in keys})

  return dict_list

# ミニバッチのデータを事例単位のリストに変換
list_dict = {
    "input_ids": [[0, 1], [2, 3]],
    "labels": [[1, 2], [3, 4]]
}

dict_list = convert_list_dict_to_dict_list(list_dict)
print(f"input: {list_dict}")
print(f"output: {dict_list}")

input: {'input_ids': [[0, 1], [2, 3]], 'labels': [[1, 2], [3, 4]]}
output: [{'input_ids': [0, 1], 'labels': [1, 2]}, {'input_ids': [2, 3], 'labels': [3, 4]}]


In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import PreTrainedModel

def run_prediction(
    dataloader: DataLoader, model: PreTrainedModel
) -> list[dict[str, any]]:
  """ 予測スコアに基づき固有表現ラベルを予測 """

  predictions = []

  # ミニバッチを処理
  for batch in tqdm(dataloader):
    inputs = {
        k: v.to(model.device)
        for k, v in batch.items()
        if k != "special_tokens_mask"
    }

    # 予測スコアを取得
    logits = model(**inputs).logits

    # 最もスコアの高いIDを取得
    batch["pred_label_ids"] = logits.argmax(-1)
    batch = {k: v.cpu().tolist() for k, v in batch.items()}

    # ミニバッチのデータを事例単位のlistに変換
    predictions += convert_list_dict_to_dict_list(batch)

  return predictions

# DataLoaderでミニバッチを作成
validation_dataloader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=data_collator
)

# 固有表現ラベルを予測する
predictions = run_prediction(validation_dataloader, model)
print(predictions[0]["pred_label_ids"]) # 最もスコアの高い予測IDを表示


100%|██████████| 17/17 [00:00<00:00, 17.84it/s]

[0, 0, 15, 16, 0, 0, 13, 14, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 14, 14, 14, 13, 0, 0, 0, 0, 0, 0, 15, 16, 0, 13, 14, 14, 14, 14, 13, 13, 14, 14, 14, 14, 0, 0, 0, 13, 14, 14, 14, 13, 0, 13, 14, 14, 14, 0, 0, 0, 0, 0, 15, 16, 16, 0, 13, 14, 14, 14, 14, 13, 0, 0, 0, 15, 15, 16, 0, 0, 13, 14]


### 固有表現の抽出

予測したラベルを示すIDの系列から表現を抽出

`predictions`に含まれる予測データを正解データの`entities`と同じフォーマット("name", "span", "type")に変換

In [ ]:
from seqeval.metrics.sequence_labeling import get_entities

def extract_entities(
    predictions: list[dict[str, any]],
    dataset: list[dict[str, any]],
    tokenizer: PreTrainedTokenizer,
    id2label: dict[int, str]
) -> list[dict[str, any]]:
  """ 固有表現を抽出 """

  results = []
  for prediction, data in zip(predictions, dataset):
    # 文字のリストを取得
    characters = list(data["text"])

    # 特殊トークンを除いたトークンのリスト、予測ラベルのリストを取得
    tokens, pred_labels = [], []

    all_tokens = tokenizer.convert_ids_to_tokens(
        prediction["input_ids"]
    )

    for token, label_id in zip(all_tokens, prediction["pred_label_ids"]):

      # 特殊トークン以外をリストに追加
      if token not in tokenizer.all_special_tokens:
        tokens.append(token)
        pred_labels.append(id2label[label_id])

    # 文字のリスト、トークンのリストのアライメントを作成
    _, token_to_char_indices = get_alignments(characters, tokens)

    # 予測ラベルのリストから固有表現タイプ、トークンの開始、終了位置を取得
    # 上記を正解データと同じ形式に変換する
    pred_entities = []
    for entity in get_entities(pred_labels):
      entity_type, token_start, token_end = entity

      # 文字単位の開始位置を取得
      char_start = token_to_char_indices[token_start][0]

      # 文字単位の終了位置を取得
      char_end = token_to_char_indices[token_end][-1] + 1
      pred_entity = {
          "name": "".join(characters[char_start:char_end]),
          "span": [char_start, char_end],
          "type": entity_type
      }

      pred_entities.append(pred_entity)
    data["pred_entities"] = pred_entities
    results.append(data)
  return results

# 固有表現を抽出
results = extract_entities(
    predictions, dataset["validation"], tokenizer, id2label
)

pprint(results[0])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'curid': '1662110',
 'entities': [{'name': '復活篇', 'span': [1, 4], 'type': '製品名'},
              {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}],
 'pred_entities': [{'name': '「', 'span': [0, 1], 'type': '_'},
                   {'name': '復活篇', 'span': [1, 4], 'type': '製品名'},
                   {'name': '」は', 'span': [4, 6], 'type': '_'},
                   {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}],
 'text': '「復活篇」はグリーンバニーからの発売となっている。'}


### 検証セットを使ったモデルの選択

最もF-scoreのマイクロ平均が高いモデルを選択する。


In [ ]:
from seqeval.metrics import classification_report

def create_character_labels(
    text: str, entities: list[dict[str, list[int] | str]]
) -> list[str]:
  """ 文字ベースでラベルのlistを作成 """

  # "0"のラベルで初期化したラベルのlistを作成
  labels = ["0"] * len(text)
  for entity in entities:
    entity_span, entity_type = entity["span"], entity["type"]

    # 固有表現の開始文字の位置に"B-"のラベルを設定
    labels[entity_span[0]] = f"B-{entity_type}"

    # 固有表現の開始文字の位置に"I-"のラベルを設定
    for i in range(entity_span[0] + 1, entity_span[1]):
      labels[i] = f"I-{entity_type}"

  return labels

def convert_results_to_labels(
    results: list[dict[str, any]]
) -> tuple[list[list[str]], list[list[str]]]:
  """ 正解データ、予測データのラベルのリスト作成 """
  true_labels, pred_labels = [], []

  for result in results:
    true_labels.append(
        create_character_labels(
            result["text"], result["entities"]
        )
    )

    pred_labels.append(
        create_character_labels(
            result["text"], result["pred_entities"]
        )
    )

  return true_labels, pred_labels

from seqeval.metrics import f1_score, precision_score, recall_score

def compute_scores(
    true_labels: list[list[str]],
    pred_labels: list[list[str]],
    average: str,
) -> dict[str, float]:
  """ precision, recall, f1_scoreを算出 """

  scores = {
      "precision": precision_score(
          true_labels, pred_labels, average=average
      ),
      "recall": recall_score(
          true_labels, pred_labels, average=average
      ),
      "f1-score": f1_score(
          true_labels, pred_labels, average=average
      )
  }

  return scores

In [ ]:
from glob import glob
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

best_score = 0

# 各チェックポイントで処理
for checkpoint in sorted(glob("/content/drive/MyDrive/Learning_LLM/chapter6/model/checkpoint-*")):
  # モデルの読み込み
  model = AutoModelForTokenClassification.from_pretrained(
      checkpoint
  )

  model.to(DEVICE)

  # 固有表現ラベルを予測
  predictions = run_prediction(validation_dataloader, model)

  # 固有表現を抽出する
  results = extract_entities(
      predictions, dataset["validation"], tokenizer, id2label
  )

  # 正解データ、予測データのラベルのリストを作成する
  true_labels, pred_labels = convert_results_to_labels(results)

  # 評価スコアを算出
  scores = compute_scores(true_labels, pred_labels, "micro")

  if best_score < scores["f1-score"]:
    best_model = model

100%|██████████| 17/17 [00:03<00:00,  5.49it/s]


In [ ]:
print(best_model)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32768, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

### 性能評価

上記で最もスコアの高かったモデルを使って、テストセットでモデルの性能を評価

In [ ]:
# テストセットに対して前処理
test_dataset = dataset["test"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id": label2id,
    },
    remove_columns = dataset["test"].column_names,
)

# DataLoaderでミニバッチを作成
test_dataloader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=data_collator
)

# 固有表現ラベルの予測
predictions = run_prediction(test_dataloader, best_model)

# 固有表現の抽出
results = extract_entities(
    predictions, dataset["test"], tokenizer, id2label
)

# 正解データ予測データのラベルのリストを作成
true_labels, pred_labels = convert_results_to_labels(results)

# 評価結果を出力
print(classification_report(true_labels, pred_labels))

Map:   0%|          | 0/535 [00:00<?, ? examples/s]

100%|██████████| 17/17 [00:03<00:00,  4.92it/s]


              precision    recall  f1-score   support

           _       0.66      0.67      0.66      1132
     その他の組織名       0.81      0.79      0.80       100
       イベント名       0.83      0.89      0.86        93
          人名       0.94      0.96      0.95       287
          地名       0.88      0.86      0.87       204
      政治的組織名       0.75      0.91      0.82       106
         施設名       0.84      0.85      0.84       137
         法人名       0.89      0.87      0.88       248
         製品名       0.77      0.82      0.80       158

   micro avg       0.76      0.78      0.77      2465
   macro avg       0.82      0.85      0.83      2465
weighted avg       0.77      0.78      0.77      2465



### エラー分析
出力の結果からなぜ固有表現をうまく抽出できなかったのか探る

In [ ]:
def find_error_results(
    results: list[dict[str, any]]
) -> list[dict[str, any]]:
  """ エラー事例を見つける """

  error_results = []
  for idx, result in enumerate(results):

    result["idx"] = idx

    # 正解データ、予測データが異なればリストに追加
    if result["entities"] != result["pred_entities"]:
      error_results.append(result)

  return error_results

def output_text_with_label(
    result: dict[str, any], entity_column: str
) -> str:
  """ 固有表現ラベル付きテキストを出力 """

  text_with_label = ""
  entity_count = 0

  for i, char in enumerate(result["text"]):
    # 出力に加えていない固有表現の有無を判定
    if entity_count < len(result[entity_column]):
      entity = result[entity_column][entity_count]

      # 固有表現の先頭の処理を行う
      if i == entity["span"][0]:
        entity_type = entity["type"]
        text_with_label += f"[({entity_type})"

      text_with_label += char

      #  固有表現の末尾の処理を行う
      if i == entity["span"][1] - 1:
        text_with_label += "]"
        entity_count += 1
    else:
      text_with_label += char

  return text_with_label

# エラーの事例を発見
error_results = find_error_results(results)

# 3件のエラー事例を出力
for result in error_results[:3]:
  idx = result["idx"]
  true_text = output_text_with_label(result, "entities")
  pred_text = output_text_with_label(result, "pred_entities")

  print(f"事例{idx}の正解：{true_text}")
  print(f"事例{idx}の予測：{pred_text}")
  print()

事例0の正解：統治機構の近代化により王朝を立て直すことに失敗、加えて[(イベント名)義和団の乱]後をめぐる[(政治的組織名)清朝]の醜態も加わり、1911年の[(イベント名)辛亥革命]への機運が高まる。
事例0の予測：[(_)統治機構の近代化により王朝を立て直すことに失敗、加えて][(イベント名)義和団の乱][(_)後をめぐる][(政治的組織名)清朝][(_)の醜態も加わり、1911年の][(イベント名)辛亥革命]への機運が高まる。

事例2の正解：[(法人名)株式会社ナムコ]に入社し、[(製品名)ファミスタ'88]等の作曲を手掛ける。
事例2の予測：[(法人名)株式会社ナムコ][(_)に入社し、][(製品名)ファミスタ'88]等の作曲を手掛ける。

事例3の正解：1947年、移行活動がまだ進行中であったため当時の[(施設名)ペイン陸軍飛行場]の軍事管理は[(政治的組織名)アメリカ陸軍航空軍]の後身である[(政治的組織名)アメリカ空軍]に移管され、空港は[(施設名)ペイン・フィールド]と改名された。
事例3の予測：[(_)1947年、移行活動がまだ進行中であったため当時の][(施設名)ペイン陸軍飛行場][(_)の軍事管理は][(政治的組織名)アメリカ陸軍航空軍][(_)の後身である][(政治的組織名)アメリカ空軍][(_)に移管され、空港は][(施設名)ペイン・フィールド]と改名された。



各トークンの予測スコアのみに基づいて固有表現のラベルを予測している
モデルの精度を向上させる
$\rightarrow$しかし、隣接したトークンの予測ラベルを考慮していないため、
間違いが起きやすい。

そこでラベル間の遷移可能性を考慮して、モデルの精度を向上させる

**ラティス**
ある文に対してラベル間の遷移可能性を考慮した固有表現ラベルの予測をすること

※考慮しない場合は予測スコアの高いラベルの経路をたどる


**ビタビアルゴリズム**

遷移する可能性のある経路の数は系列長に対して指数関数敵に増加するため効率が悪い。
そこで効率よくラベル列を予測する方法として重宝される



### 遷移スコアを定義

In [ ]:
def create_transitions(
    label2id: dict[str, int]
) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:

  """ 遷移スコアを定義 """
  # B-のラベルIDのリスト
  b_ids = [v for k, v in label2id.items() if k[0] == "B"]

  # I-のラベルIDのリスト
  i_ids = [v for k, v in label2id.items() if k[0] == "I"]

  # OのラベルID
  o_id = label2id["0"]

  # 開始遷移スコアを定義
  # すべてのスコアを-100で初期化
  start_transitions = torch.full([len(label2id)], -100.0)
  # すべてのラベルからB-へ遷移可能としてOを代入
  start_transitions[b_ids] = 0

  # すべてのラベルからOへ遷移可能としてOを代入
  start_transitions[o_id] = 0


  # ラベル間の遷移スコアを定義
  # 全てのスコアを-100で初期化
  transitions = torch.full([len(label2id), len(label2id)], -100.0)

  # すべてのラベルからB-へ遷移可能としてOを代入
  transitions[:, b_ids] = 0

  # すべてのラベルからOへ遷移可能としてOを代入
  transitions[:, o_id] = 0

  # B-から同じタイプのI-へ遷移可能としてOを代入
  transitions[b_ids, i_ids] = 0

  #I-から同じタイプのI-への遷移可能ととして0を代入
  transitions[i_ids, i_ids] = 0

  # 終了遷移スコアを定義
  # すべてのラベルから遷移可能としてすべてのスコアを0とする
  end_transitions = torch.zeros(len(label2id))
  return start_transitions, transitions, end_transitions

# 遷移スコアを定義
start_transitions, transitions, end_transitions = create_transitions(label2id)


In [ ]:
# 出力を確認する
print(start_transitions)
print(transitions)
print(end_transitions)

tensor([   0.,    0., -100.,    0., -100.,    0., -100.,    0., -100.,    0.,
        -100.,    0., -100.,    0., -100.,    0., -100.])
tensor([[   0.,    0., -100.,    0., -100.,    0., -100.,    0., -100.,    0.,
         -100.,    0., -100.,    0., -100.,    0., -100.],
        [   0.,    0.,    0.,    0., -100.,    0., -100.,    0., -100.,    0.,
         -100.,    0., -100.,    0., -100.,    0., -100.],
        [   0.,    0.,    0.,    0., -100.,    0., -100.,    0., -100.,    0.,
         -100.,    0., -100.,    0., -100.,    0., -100.],
        [   0.,    0., -100.,    0.,    0.,    0., -100.,    0., -100.,    0.,
         -100.,    0., -100.,    0., -100.,    0., -100.],
        [   0.,    0., -100.,    0.,    0.,    0., -100.,    0., -100.,    0.,
         -100.,    0., -100.,    0., -100.,    0., -100.],
        [   0.,    0., -100.,    0., -100.,    0.,    0.,    0., -100.,    0.,
         -100.,    0., -100.,    0., -100.,    0., -100.],
        [   0.,    0., -100.,    0.,

### ビタビアルゴリズムを用いたラベル列の予測

In [ ]:
def decode_with_viterbi(
    emissions: torch.Tensor, # ラベルの予測スコア
    mask: torch.Tensor, # マスク
    start_transitions: torch.Tensor, # 開始遷移スコア
    transitions: torch.Tensor, # ラベル間の遷移スコア
    end_transitions: torch.Tensor # 終了遷移スコア
) -> torch.Tensor:
  """ ビタビアルゴリズムを用いて最適なラベル列を探索 """

  # バッチサイズ、系列長を取得
  batch_size, seq_length = mask.shape

  # 予測スコア、マスクに0次元目と1次元目を入れ替え
  emissions = emissions.transpose(1, 0)
  mask = mask.transpose(1, 0)

  histories = []

  # 開始遷移スコア、予測スコアを加算。累積スコアの初期値とする
  score = start_transitions + emissions[0]

  for i in range(1, seq_length):
    # 累積スコアを3次元に変換
    broad_cast_score = score.unsqueeze(2)

    # 現在の予測スコアを3次元に変換
    broadcast_emission = emissions[i].unsqueeze(1)

    # 現在の累積スコアを取得
    next_score = (broad_cast_score + transitions + broadcast_emission)

    # 現在の累積スコアの各ラベルの最大値、インデックスを取得
    next_score, indices = next_score.max(dim=1)

    # マスクしない要素なら累積スコアを更新
    score = torch.where(mask[i].unsqueeze(1), next_score, score)

    # スコアの高いインデックスを履歴のリストに追加
    histories.append(indices)

  # 終了遷移スコアを加算、合計スコアにする
  score += end_transitions

  # 各事例で最適なラベル列を取得
  best_labels_list = []
  for i in range(batch_size):

    # 合計スコアの中で最大のスコアとなるラベルを取得
    _, best_last_label = score[i].max(dim=0)
    best_labels = [best_last_label.item()]

    # 最後のラベルの遷移を逆方向に探索、最適なラベル列を取得
    for history in reversed(histories):
      best_last_label = history[i][best_labels[-1]]
      best_labels.append(best_last_label.item())

    # 順序を反転
    best_labels.reverse()
    best_labels_list.append(best_labels)

  return torch.LongTensor(best_labels_list)

In [ ]:
def run_prediction_viterbi(
    dataloader: DataLoader,
    model: PreTrainedModel,
) -> list[dict[str, any]]:
  """ ビタビアルゴリズムを用いてラベルを予測 """

  # 遷移スコアを取得
  start_transitions, transitions, end_transitions = (
      create_transitions(model.config.label2id)
  )

  predictions = []
  for batch in tqdm(dataloader):
    inputs = {
        k: v.to(model.device)
        for k, v in batch.items()
        if k != "special_tokens_mask"
    }

    # [CLS]以外の予測スコアを取得
    logits = model(**inputs).logits.cpu()[:, 1:, :]

    # [CLS]以外の特殊トークンのマスクを取得
    mask = (batch["special_tokens_mask"].cpu() == 0)[:, 1:]

    # ビタビアルゴリズムを用いて最適なIDの系列を探索
    pred_label_ids = decode_with_viterbi(
        logits,
        mask,
        start_transitions,
        transitions,
        end_transitions
    )

    # [CLS]のIDを0とする
    cls_pred_label_id = torch.zeros(pred_label_ids.shape[0], 1)

    # [CLS]のIDと、探索したIDの系列を連結して予測ラベルとする
    batch["pred_label_ids"] = torch.concat(
        [cls_pred_label_id, pred_label_ids], dim=1
    )

    batch = {k: v.cpu().tolist() for k, v in batch.items()}

    # ミニバッチのデータを事例単位のlistに変換
    predictions += convert_list_dict_to_dict_list(batch)

  return predictions

# ビタビアルゴリズムを用いてラベルを予測
predictions = run_prediction_viterbi(test_dataloader, best_model)

# 固有表現を抽出
results = extract_entities(
    predictions, dataset["test"], tokenizer, id2label
)

# 正解データ、予測データのラベルのlistを作成
true_labels, pred_labels = convert_results_to_labels(results)

# 評価結果を出力
print(classification_report(true_labels, pred_labels))



100%|██████████| 17/17 [00:03<00:00,  4.51it/s]
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


              precision    recall  f1-score   support

           _       0.66      0.67      0.66      1132
     その他の組織名       0.82      0.79      0.81       100
       イベント名       0.86      0.89      0.87        93
          人名       0.94      0.96      0.95       287
          地名       0.89      0.87      0.88       204
      政治的組織名       0.79      0.92      0.85       106
         施設名       0.86      0.85      0.85       137
         法人名       0.89      0.88      0.88       248
         製品名       0.80      0.82      0.81       158

   micro avg       0.77      0.78      0.78      2465
   macro avg       0.83      0.85      0.84      2465
weighted avg       0.77      0.78      0.78      2465



In [ ]:
results[27]

{'curid': '62926',
 'text': '李承晩政権期から朴正煕政権期の1970年前後まで、南側の大韓民国よりも北側の朝鮮民主主義人民共和国の方が経済的な体力では勝っていたのである。',
 'entities': [{'name': '李承晩政権', 'span': [0, 5], 'type': '政治的組織名'},
  {'name': '朴正煕政権', 'span': [8, 13], 'type': '政治的組織名'},
  {'name': '大韓民国', 'span': [28, 32], 'type': '地名'},
  {'name': '朝鮮民主主義人民共和国', 'span': [38, 49], 'type': '地名'}],
 'pred_entities': [{'name': '李承晩', 'span': [0, 3], 'type': '人名'},
  {'name': '政権期から', 'span': [3, 8], 'type': '_'},
  {'name': '朴正煕', 'span': [8, 11], 'type': '人名'},
  {'name': '政権期の1970年前後まで、南側の', 'span': [11, 28], 'type': '_'},
  {'name': '大韓民国', 'span': [28, 32], 'type': '地名'},
  {'name': 'よりも北側の', 'span': [32, 38], 'type': '_'},
  {'name': '朝鮮民主主義人民共和国', 'span': [38, 49], 'type': '地名'}]}

In [ ]:
idx = 27
result = results[idx]
true_text = output_text_with_label(result, "entities")
pred_text = output_text_with_label(result, "pred_entities")
print(f"事例{idx}の正解: {true_text}")
print(f"事例{idx}の予測: {pred_text}")

事例27の正解: [(政治的組織名)李承晩政権]期から[(政治的組織名)朴正煕政権]期の1970年前後まで、南側の[(地名)大韓民国]よりも北側の[(地名)朝鮮民主主義人民共和国]の方が経済的な体力では勝っていたのである。
事例27の予測: [(人名)李承晩][(_)政権期から][(人名)朴正煕][(_)政権期の1970年前後まで、南側の][(地名)大韓民国][(_)よりも北側の][(地名)朝鮮民主主義人民共和国]の方が経済的な体力では勝っていたのである。
